<a href="https://colab.research.google.com/github/Baah134/Baah134/blob/main/SER_CARINE/Paper_6_Wac2Vec%2BFFN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# --- 1. Install Necessary Libraries ---
!pip install transformers datasets evaluate soundfile librosa torchaudio accelerate pandas -q

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import librosa
import numpy as np
from sklearn.model_selection import train_test_split
from transformers import AutoFeatureExtractor, AutoModel
from google.colab import drive
from tqdm.auto import tqdm

# **ARC LAB**

In [ ]:
import zipfile
import os

# 1. Define the path to your zip file
zip_file_path = '/content/actual_3.zip'  # <--- IMPORTANT: Change this to your file's path

# 2. Define the directory where you want to extract the files
extract_dir = 'actual'

# 3. Create the target directory if it doesn't already exist
os.makedirs(extract_dir, exist_ok=True)

# 4. Open the zip file and extract its contents
try:
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(extract_dir)

    print(f"Successfully unzipped '{zip_file_path}' into the '{extract_dir}' folder.")

except zipfile.BadZipFile:
    print(f"Error: The file '{zip_file_path}' is not a valid zip file or is corrupted.")
except FileNotFoundError:
    print(f"Error: The file '{zip_file_path}' was not found.")
except Exception as e:
    print(f"An error occurred: {e}")

KeyboardInterrupt: 

In [ ]:
import pandas as pd
import os

# Define Paths
CSV_RUNTIME_PATH = "/content/converted_emotions.csv"

RUNTIME_AUDIO_FOLDER = "/content/actual/actual3"
NEW_CSV_RUNTIME_PATH = "/content/updated_emotions_runtime.csv"



# Load the CSV
try:
    df = pd.read_csv(CSV_RUNTIME_PATH)
    print("Successfully loaded emotions.csv.")
    print("Original DataFrame head:")
    display(df.head())
except FileNotFoundError:
    print(f"Error: CSV file not found at {CSV_RUNTIME_PATH}. Please ensure you have uploaded 'emotions.csv' to your runtime.")
    exit()
except Exception as e:
    print(f"An error occurred while loading the CSV: {e}")
    exit()

# 3. Update File Paths
def update_runtime_filepath(filename):
    base_filename = os.path.basename(filename)
    new_filepath = os.path.join(RUNTIME_AUDIO_FOLDER, base_filename)
    return new_filepath

try:
    df['filename'] = df['filename'].apply(update_runtime_filepath)
    print("File paths in the DataFrame updated.")
    print("DataFrame head with updated paths:")
    display(df.head())
except KeyError:
    print("Error: 'filename' column not found in the DataFrame. Please check your CSV file.")
    exit()
except Exception as e:
    print(f"An unexpected error occurred while updating DataFrame paths: {e}")
    exit()

# 4. Save New CSV

try:
    df.to_csv(NEW_CSV_RUNTIME_PATH, index=False)
    print(f"Successfully saved updated CSV to {NEW_CSV_RUNTIME_PATH}")
except Exception as e:
    print(f"An error occurred while saving the new CSV: {e}")
    exit()

Successfully loaded emotions.csv.
Original DataFrame head:


,filename,emotion
0,/content/drive/MyDrive/DeepLearning/SER/ACE/ac...,Joy
1,/content/drive/MyDrive/DeepLearning/SER/ACE/ac...,Neutral
2,/content/drive/MyDrive/DeepLearning/SER/ACE/ac...,Joy
3,/content/drive/MyDrive/DeepLearning/SER/ACE/ac...,Neutral
4,/content/drive/MyDrive/DeepLearning/SER/ACE/ac...,Neutral


File paths in the DataFrame updated.
DataFrame head with updated paths:


,filename,emotion
0,/content/actual/actual3/kae_58_64.wav,Joy
1,/content/actual/actual3/kae_78_81.wav,Neutral
2,/content/actual/actual3/kae_83_90.wav,Joy
3,/content/actual/actual3/kae_107_110.wav,Neutral
4,/content/actual/actual3/kae_111_118.wav,Neutral


Successfully saved updated CSV to /content/updated_emotions_runtime.csv


In [ ]:
CSV_PATH = "/content/updated_emotions_runtime.csv"

## **ARC LAB RAVDESS**

In [ ]:
#Train
ravdess ="/content/drive/MyDrive/DeepLearning/External/RAVDESS Emotional Speech Audio/audio_speech_actors_01-24/"
ravdess_directory_list = os.listdir(ravdess)
print(ravdess_directory_list)

file_emotion = []
file_path = []
for i in ravdess_directory_list:
    # as their are 24 different actors in our previous directory we need to extract files for each actor.
    actor = os.listdir(ravdess + i)
    for f in actor:
        part = f.split('.')[0].split('-')
    # third part in each file represents the emotion associated to that file.
        file_emotion.append(int(part[2]))
        file_path.append(ravdess + i + '/' + f)

# dataframe for emotion of files
emotion_df = pd.DataFrame(file_emotion, columns=['Emotions'])
# dataframe for path of files.
path_df = pd.DataFrame(file_path, columns=['Path'])
ravdess_df = pd.concat([emotion_df, path_df], axis=1)
# changing integers to actual emotions.
ravdess_df.Emotions.replace({1:'neutral', 2:'neutral', 3:'happy', 4:'sad', 5:'angry', 6:'fear', 7:'disgust',
                             8:'surprise'},
                            inplace=True)
print(ravdess_df.head())
print("______________________________________________")
print(ravdess_df.tail())
print("_______________________________________________")
print(ravdess_df.Emotions.value_counts())

# Save the processed DataFrame to a CSV file
ravdess_df.to_csv('/content/ravdess_processed.csv', index=False)

# **GOOGLE DRIVE**

In [ ]:
#Mount Google Drive

drive.mount('/content/drive')

CSV_PATH ="/content/drive/MyDrive/DeepLearning/SER/ACE/converted_emotions.csv"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os
import pandas as pd

# Path to data
ravdess = "/content/drive/MyDrive/DeepLearning/External/RAVDESS Emotional Speech Audio/audio_speech_actors_01-24/"
ravdess_directory_list = os.listdir(ravdess)
print(ravdess_directory_list)

file_emotion = []
file_path = []
file_speaker = []  # <--- NEW LIST for Speaker IDs

for i in ravdess_directory_list:
    # as there are 24 different actors in our previous directory we need to extract files for each actor.
    actor_path = os.path.join(ravdess, i)

    # Check if it is a directory to avoid errors with random files
    if os.path.isdir(actor_path):
        actor_files = os.listdir(actor_path)

        for f in actor_files:
            # Filename example: 03-01-06-01-02-01-12.wav
            part = f.split('.')[0].split('-')

            # 3rd part (index 2) is Emotion
            file_emotion.append(int(part[2]))

            # 7th part (index 6) is Actor ID <--- EXTRACTION HERE
            file_speaker.append(int(part[6]))

            file_path.append(os.path.join(actor_path, f))

# DataFrame for emotion
emotion_df = pd.DataFrame(file_emotion, columns=['emotion'])

# DataFrame for paths
path_df = pd.DataFrame(file_path, columns=['path'])

# DataFrame for speakers <--- NEW COLUMN
speaker_df = pd.DataFrame(file_speaker, columns=['speaker'])

# Concatenate all
ravdess_df = pd.concat([emotion_df, speaker_df, path_df], axis=1)

# Changing integers to actual emotions
ravdess_df.emotion.replace({1:'neutral', 2:'neutral', 3:'happy', 4:'sad', 5:'angry', 6:'fear', 7:'disgust', 8:'surprise'}, inplace=True)

print(ravdess_df.head())
print("______________________________________________")
print(ravdess_df.tail())
print("_______________________________________________")
print(ravdess_df.emotion.value_counts())
print("\nSpeaker Counts:")
print(ravdess_df['speaker'].value_counts().sort_index()) # Check if you have all 24 actors

# Save the processed DataFrame with Speakers to CSV
ravdess_df.to_csv('/content/ravdess_processed.csv', index=False)

['Actor_07', 'Actor_09', 'Actor_08', 'Actor_06', 'Actor_01', 'Actor_24', 'Actor_13', 'Actor_22', 'Actor_03', 'Actor_05', 'Actor_14', 'Actor_15', 'Actor_12', 'Actor_02', 'Actor_23', 'Actor_20', 'Actor_21', 'Actor_17', 'Actor_10', 'Actor_04', 'Actor_11', 'Actor_16', 'Actor_19', 'Actor_18']
   emotion  speaker                                               path
0    happy        7  /content/drive/MyDrive/DeepLearning/External/R...
1      sad        7  /content/drive/MyDrive/DeepLearning/External/R...
2     fear        7  /content/drive/MyDrive/DeepLearning/External/R...
3  disgust        7  /content/drive/MyDrive/DeepLearning/External/R...
4    angry        7  /content/drive/MyDrive/DeepLearning/External/R...
______________________________________________
       emotion  speaker                                               path
1435     angry       18  /content/drive/MyDrive/DeepLearning/External/R...
1436      fear       18  /content/drive/MyDrive/DeepLearning/External/R...
1437       sa

/tmp/ipython-input-1733908336.py:46: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  ravdess_df.emotion.replace({1:'neutral', 2:'neutral', 3:'happy', 4:'sad', 5:'angry', 6:'fear', 7:'disgust', 8:'surprise'}, inplace=True)


# **CONTINUATION**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split

# ==========================================
# 1. CONFIGURATION
# ==========================================
CSV_PATH = '/content/ravdess_processed.csv'

# STRICT SPLIT SETTINGS
# Speakers to reserve for testing in Phase 2
TEST_SPEAKERS_STRICT = [21, 22, 23, 24]

# ==========================================
# 2. DATA LOADING & SPLITTING LOGIC
# ==========================================
def load_data_and_split(mode='random'):
    """
    Args:
        mode: 'random' (for verification) or 'strict' (for hypothesis testing)
    Returns:
        train_df, val_df (mapped from test_df for compatibility)
    """
    print(f"Loading data from {CSV_PATH}...")
    if not os.path.exists(CSV_PATH):
        print("Error: CSV not found. Run extraction script first.")
        return None, None

    df = pd.read_csv(CSV_PATH)

    # Compatibility: Rename columns to match your code's expectations
    # 'Emotions' -> 'emotion', 'Path' -> 'path'
    df.rename(columns={'Emotions': 'emotion', 'Path': 'path', 'Speaker': 'speaker'}, inplace=True)

    # --- MODE 1: RANDOM SPLIT (Verification) ---
    if mode == 'random':
        print("--- RUNNING VERIFICATION (RANDOM SPLIT) ---")
        print("Logic: Shuffling all files and taking 20% for validation.")

        # Simple random split
        train_df, val_df = train_test_split(df, test_size=0.2, random_state=42, shuffle=True)

    # --- MODE 2: STRICT SPEAKER SPLIT (Hypothesis) ---
    elif mode == 'strict':
        print("--- RUNNING STRICT SPLIT (SPEAKER INDEPENDENT) ---")
        print(f"Logic: Training on Speakers 1-20. Validation ONLY on {TEST_SPEAKERS_STRICT}.")

        # Boolean Masking based on 'speaker' column
        val_mask = df['speaker'].isin(TEST_SPEAKERS_STRICT)

        val_df = df[val_mask].copy()
        train_df = df[~val_mask].copy() # The symbol '~' means NOT

    else:
        raise ValueError("Mode must be 'random' or 'strict'")

    return train_df, val_df

# ==========================================
# 3. USAGE EXAMPLE & MERGED LOGIC
# ==========================================
if __name__ == "__main__":
    # USER: CHANGE THIS VARIABLE TO SWITCH MODES
    # 'random' = Verify code works/high accuracy
    # 'strict' = Test your research hypothesis (Accuracy should drop)
    CURRENT_MODE = 'strict'

    # --- 4. Load Data & Split (Using Strict Logic) ---
    train_df, val_df = load_data_and_split(CURRENT_MODE)

    if train_df is not None:
        print(f"\nData split ({CURRENT_MODE}): {len(train_df)} training, {len(val_df)} validation")

        # Verify Speakers in Validation Set
        print("Validation Speakers:", val_df['speaker'].unique())

        # Recombine for label extraction (to ensure we capture all possible labels)
        df = pd.concat([train_df, val_df])

        # --- 5. Create Label Mappings (Your Code) ---
        print("\nCreating label mappings...")
        label_list = sorted(df["emotion"].unique())
        num_labels = len(label_list)

        label2id = {label: i for i, label in enumerate(label_list)}
        id2label = {i: label for label, i in label2id.items()} # Useful for inference later

        print(f"Found {num_labels} labels: {label_list}")
        print(f"Label Map: {label2id}")

        # You now have 'train_df', 'val_df', 'label2id', and 'num_labels' ready for your model loop.

In [ ]:
# --- 3. Define Your Model & Data Paths ---
print("Setting up model and data paths...")


# --- 4. Load Data with Pandas & Split ---
print(f"Loading dataset from {CSV_PATH} using pandas...")
df = pd.read_csv(CSV_PATH)
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42, shuffle=True)
print(f"Data split: {len(train_df)} training, {len(val_df)} validation")

# --- 5. Create Label Mappings ---
print("Creating label mappings...")
label_list = sorted(df["emotion"].unique())
num_labels = len(label_list) # This should be 4
label2id = {label: i for i, label in enumerate(label_list)}
print(f"Found {num_labels} labels: {label_list}")

Setting up model and data paths...
Loading dataset from /content/drive/MyDrive/DeepLearning/SER/ACE/converted_emotions.csv using pandas...
Data split: 750 training, 188 validation
Creating label mappings...
Found 4 labels: ['Anger', 'Joy', 'Neutral', 'Sadness']


In [ ]:
import torch # Ensure torch is imported
from transformers import AutoFeatureExtractor, AutoModel

MODEL_NAME = "facebook/wav2vec2-base"

print(f"Loading feature extractor for {MODEL_NAME}...")
feature_extractor = AutoFeatureExtractor.from_pretrained(MODEL_NAME)
print("Feature extractor loaded.")

print(f"Loading full Wav2Vec2 model for {MODEL_NAME}...")
# Load the full model
wav2vec2_model = AutoModel.from_pretrained(MODEL_NAME)

# Move the model to GPU if available and set to evaluation mode
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
wav2vec2_model.to(device)

print(f"Full Wav2Vec2 model loaded and moved to {device}.")

Loading feature extractor for facebook/wav2vec2-base...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/transformers/configuration_utils.py:335: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


Feature extractor loaded.
Loading full Wav2Vec2 model for facebook/wav2vec2-base...
Full Wav2Vec2 model loaded and moved to cuda.


# **OLD**

In [ ]:
import tensorflow as tf
import numpy as np
import librosa
from tqdm.auto import tqdm
import torch
import random # Import random for augmentation probability


print("Building tf.data pipelines with Mean Pooling and Reshaping...")


TARGET_SAMPLING_RATE = 16000


MAX_LENGTH = feature_extractor.sampling_rate * 5 # Assuming 5 seconds

print(f"Target sampling rate for processing: {TARGET_SAMPLING_RATE} Hz")
print(f"Feature extractor configured max_length: {MAX_LENGTH}")

# Define the batch size for the dataset pipeline
BATCH_SIZE = 32 # You can adjust this based on your GPU memory

print(f"Using batch size: {BATCH_SIZE}")

# Get the device for model inference
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# --- Data Augmentation Functions ---

# Noise augmentation (User's Code)
def add_noise(data):
    noise_amp = 0.035 * np.random.uniform() * np.amax(data)
    data = data + noise_amp * np.random.normal(size=data.shape[0])
    return data

# Pitch augmentation (User's Code)
def pitch_shift(data, sampling_rate, pitch_factor=0.7):
    return librosa.effects.pitch_shift(y=data, sr=sampling_rate, n_steps=pitch_factor)

# Time Shifting augmentation (My Code)
def time_shift(data, sampling_rate, shift_limit_ms=100):
    """Shifts audio data in time."""
    shift_limit_samples = int(sampling_rate * shift_limit_ms / 1000)
    shift_amount = random.randint(-shift_limit_samples, shift_limit_samples)
    return np.roll(data, shift_amount)


# --- Create a Generator Function ---
# This function will yield one sample at a time from the dataframe
def data_generator(dataframe, feature_extractor, wav2vec2_model, label2id, target_sampling_rate, max_length, device, is_training=True):

    # Set the model to evaluation mode (should already be done, but good practice)
    wav2vec2_model.eval()

    for index, row in dataframe.iterrows():
        audio_path = row['filename']
        emotion_label = row['emotion']
        label_id = label2id[emotion_label]
        label_id_np = np.array(label_id, dtype=np.int32)

        try:
            # Load Audio AND Resample to TARGET_SAMPLING RATE
            audio_data, sampling_rate = librosa.load(audio_path, sr=target_sampling_rate)

            # List to hold original and augmented data
            augmented_data_list = [audio_data] # Start with the original data

            # Apply all three augmentations for training data and add to the list
            if is_training:
                # Apply noise augmentation
                augmented_data_list.append(add_noise(audio_data.copy())) # Use .copy() to avoid modifying original

                # Apply pitch shift augmentation
                # Randomly choose a pitch shift factor for pitch shifting
                pitch_factor_rand = random.uniform(-4, 4) # Shift between -4 and +4 semitones
                augmented_data_list.append(pitch_shift(audio_data.copy(), sampling_rate, pitch_factor_rand))

                # Apply time shift augmentation
                # Randomly choose a time shift limit (e.g., up to 200ms)
                time_shift_limit_ms = random.uniform(50, 200) # Shift between 50ms and 200ms
                augmented_data_list.append(time_shift(audio_data.copy(), sampling_rate, time_shift_limit_ms))


            # Process each item in the augmented_data_list
            for current_audio_data in augmented_data_list:
                # Apply Feature Extractor
                features = feature_extractor(
                    current_audio_data, # Use the current audio data (original or augmented)
                    sampling_rate=sampling_rate,
                    padding='max_length',
                    max_length=max_length,
                    truncation=True,
                    return_tensors="pt"          # Get PyTorch tensor output
                )
                input_values = features['input_values'].to(device) # Move input to device

                # Pass input through the full Wav2Vec2 model
                with torch.no_grad(): # Gradients aren't needed for feature extraction
                    model_outputs = wav2vec2_model(input_values)

                # Extract the last hidden state (rich features)
                # The shape will be (batch_size=1, sequence_length', hidden_size)
                raw_features = model_outputs.last_hidden_state

                # Convert PyTorch tensor to NumPy and squeeze the batch dimension (1,)
                # The shape is now (sequence_length', hidden_size), e.g., (249, 1024)
                processed_features_np = raw_features.squeeze(0).cpu().numpy().astype(np.float32)

                # --- Apply Mean Pooling ---
                # Apply mean pooling along the original sequence length dimension (axis=0 in NumPy after squeezing batch)
                # Shape (sequence_length', hidden_size) -> (hidden_size,)
                pooled_features_np = np.mean(processed_features_np, axis=0)

                # --- Reshape for 1D CNN Input ---
                # Reshape the (hidden_size,) vector to (hidden_size, 1) for Conv1D with 1 channel
                # Shape (hidden_size,) -> (hidden_size, 1), e.g., (1024,) -> (1024, 1)
                reshaped_features_np = np.expand_dims(pooled_features_np, axis=-1)


                # Get Label ID
                label_id = label2id[emotion_label]
                label_id_np = np.array(label_id, dtype=np.int32)

                # Yield the reshaped pooled feature array and label ID
                yield reshaped_features_np, label_id_np

        except FileNotFoundError:
            print(f"Warning: Audio file not found at {audio_path}. Skipping.")
            continue # Skip to the next sample
        except Exception as e:
            print(f"Warning: An error occurred processing {audio_path}: {e}. Skipping.")
            continue # Skip to the next sample


# Build the tf.data.Dataset using the generator
def create_tf_dataset(dataframe, feature_extractor, wav2vec2_model, label2id, target_sampling_rate, max_length, batch_size, device, feature_shape_after_pooling, is_training=True):

    print(f"Using pooled and reshaped Wav2Vec2 output feature shape per sample: {feature_shape_after_pooling}")

    # Calculate the number of samples generated per original row
    samples_per_row = 4 if is_training else 1

    dataset = tf.data.Dataset.from_generator(
        lambda: data_generator(dataframe, feature_extractor, wav2vec2_model, label2id, target_sampling_rate, max_length, device, is_training),
        output_signature=(
            # Output shape is now (hidden_size, 1) after mean pooling and reshaping
            tf.TensorSpec(shape=feature_shape_after_pooling, dtype=tf.float32),
            tf.TensorSpec(shape=(), dtype=tf.int32)
        )
    )

    if is_training:
        # Shuffle the dataset (buffer_size should now be based on the augmented size)
        augmented_dataset_size = len(dataframe) * samples_per_row
        dataset = dataset.shuffle(buffer_size=augmented_dataset_size)
        # Add .repeat() for training dataset to iterate over multiple epochs
        dataset = dataset.repeat()


    # Batch the dataset
    dataset = dataset.batch(batch_size)


    # Prefetch data to overlap loading and training
    dataset = dataset.prefetch(buffer_size=tf.data.AUTOTUNE)

    return dataset


# Determine the new feature shape after pooling and reshaping
# Assuming the hidden_size of Wav2Vec2 is 1024 based on previous runs
# --- FIX #1: Correctly define pooled_feature_dimension ---
# Do not assume 1024. Get the dimension directly from the model's config.
try:
    # Ensure wav2vec2_model is loaded before accessing its config
    if 'wav2vec2_model' in globals() and wav2vec2_model is not None:
         pooled_feature_dimension = wav2vec2_model.config.hidden_size
         print(f"Successfully set pooled_feature_dimension from model config: {pooled_feature_dimension}")
    else:
        print("Warning: wav2vec2_model not found. Assuming pooled_feature_dimension is 1024. This may cause errors.")
        pooled_feature_dimension = 1024 # Fallback if model not loaded
except Exception as e:
    print(f"Error getting hidden_size from model config: {e}")
    print("Assuming 1024, but this may cause errors later.")
    pooled_feature_dimension = 1024


feature_shape_after_pooling = (pooled_feature_dimension, 1) # Shape is (hidden_size, 1)

# Create the training and validation tf.data.Dataset objects
# Pass is_training=True for the training dataset to enable augmentation
train_dataset = create_tf_dataset(train_df, feature_extractor, wav2vec2_model, label2id, TARGET_SAMPLING_RATE, MAX_LENGTH, BATCH_SIZE, device, feature_shape_after_pooling, is_training=True)
val_dataset = create_tf_dataset(val_df, feature_extractor, wav2vec2_model, label2id, TARGET_SAMPLING_RATE, MAX_LENGTH, BATCH_SIZE, device, feature_shape_after_pooling, is_training=False)


print("tf.data pipelines with Mean Pooling and Reshaping are ready.")


# Optional: Take a batch to verify the shape and data types
try:
     for batch_features, batch_labels in train_dataset.take(1):
          print("\nSample Batch Features Shape:", batch_features.shape)
          print("Sample Batch Labels Shape:", batch_labels.shape)
          print("Sample Batch Features Dtype:", batch_features.dtype)
          print("Sample Batch Labels Dtype:", batch_labels.dtype)
          break # Only take one batch for inspection
except Exception as e:
    print(f"\nCould not verify batch shapes. Error: {e}")
    print("This might happen if the train_dataset is empty or data loading failed.")

Building tf.data pipelines with Mean Pooling and Reshaping...
Target sampling rate for processing: 16000 Hz
Feature extractor configured max_length: 80000
Using batch size: 32
Successfully set pooled_feature_dimension from model config: 768
Using pooled and reshaped Wav2Vec2 output feature shape per sample: (768, 1)
Using pooled and reshaped Wav2Vec2 output feature shape per sample: (768, 1)
tf.data pipelines with Mean Pooling and Reshaping are ready.


In [ ]:
import os
import numpy as np
import librosa
import torch
from tqdm.auto import tqdm
import pandas as pd
import tensorflow as tf # Import TensorFlow for tf.data

print("Starting feature extraction and saving...")

# Define output directories for saving features and labels
TRAIN_FEATURES_DIR = "/content/train_features"
VAL_FEATURES_DIR = "/content/val_features"

os.makedirs(TRAIN_FEATURES_DIR, exist_ok=True)
os.makedirs(VAL_FEATURES_DIR, exist_ok=True)

if 'TARGET_SAMPLING_RATE' not in globals():
    TARGET_SAMPLING_RATE = 16000
    print(f"TARGET_SAMPLING_RATE not found, setting to default: {TARGET_SAMPLING_RATE}")
if 'MAX_LENGTH' not in globals():
    # Assuming feature_extractor is available to get sampling_rate
    if 'feature_extractor' in globals() and feature_extractor is not None:
        MAX_LENGTH = feature_extractor.sampling_rate * 5 # Assuming 5 seconds
    else:
        MAX_LENGTH = 16000 * 5 # Default if feature_extractor is not available
        print("Warning: feature_extractor not found, setting default MAX_LENGTH.")
    print(f"MAX_LENGTH not found, setting to default: {MAX_LENGTH}")

if 'feature_extractor' not in globals() or feature_extractor is None:
    print("Error: feature_extractor is not defined. Please run the cell that defines it.")
    # You might want to exit or raise an error here if critical
    exit()
if 'wav2vec2_model' not in globals() or wav2vec2_model is None:
    print("Error: wav2vec2_model is not defined. Please run the cell that defines it.")
    exit()
if 'label2id' not in globals():
    print("Error: label2id is not defined. Please run the cell that defines it.")
    exit()
if 'device' not in globals():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Device not previously defined, setting to: {device}")

# --- FIX #1: Correctly define pooled_feature_dimension ---
# Do not assume 1024. Get the dimension directly from the model's config.
try:
    pooled_feature_dimension = wav2vec2_model.config.hidden_size
    print(f"Successfully set pooled_feature_dimension from model config: {pooled_feature_dimension}")
except Exception as e:
    print(f"Error getting hidden_size from model config: {e}")
    print("Assuming 1024, but this may cause errors later.")
    pooled_feature_dimension = 1024


# Ensure dataframes are loaded
if 'train_df' not in globals() or 'val_df' not in globals():
    print("Error: train_df or val_df not found. Please run the cell that loads and splits the data.")
    exit()


# Set the Wav2Vec2 model to evaluation mode
wav2vec2_model.eval()

def extract_and_save_features(dataframe, output_dir, feature_extractor, wav2vec2_model, label2id, target_sampling_rate, max_length, device):
    """
    Extracts features using Wav2Vec2 with mean pooling and saves them to disk.
    """
    print(f"Extracting features for {len(dataframe)} samples into {output_dir}...")

    # Using enumerate is slightly safer than relying on dataframe.iterrows() index
    for i, (index, row) in enumerate(tqdm(dataframe.iterrows(), total=len(dataframe), desc="Extracting Features")):
        audio_path = row['filename']
        emotion_label = row['emotion']
        unique_id = f"{i:05d}" # Create a unique ID for saving files based on enumeration

        try:
            # Load Audio AND Resample to TARGET_SAMPLING_RATE
            audio_data, sampling_rate = librosa.load(audio_path, sr=target_sampling_rate)

            # Apply Feature Extractor
            features = feature_extractor(
                audio_data,
                sampling_rate=sampling_rate,
                padding='max_length',
                max_length=max_length,
                truncation=True,
                return_tensors="pt"          # Get PyTorch tensor output
            )
            input_values = features['input_values'].to(device) # Move input to device

            # Pass input through the full Wav2Vec2 model
            with torch.no_grad(): # Gradients aren't needed for feature extraction
                model_outputs = wav2vec2_model(input_values)

            # Extract the last hidden state (rich features)
            raw_features = model_outputs.last_hidden_state # Shape (batch_size=1, sequence_length', hidden_size)

            # Convert PyTorch tensor to NumPy and squeeze the batch dimension (1,)
            processed_features_np = raw_features.squeeze(0).cpu().numpy().astype(np.float32) # Shape (sequence_length', hidden_size)

            # --- Apply Mean Pooling ---
            # Apply mean pooling along the original sequence length dimension (axis=0 in NumPy)
            pooled_features_np = np.mean(processed_features_np, axis=0) # Shape (hidden_size,)

            # --- Reshape for 1D CNN Input before saving ---
            # Reshape the (hidden_size,) vector to (hidden_size, 1) for Conv1D with 1 channel
            # Shape (hidden_size,) -> (hidden_size, 1), e.g., (1024,) -> (1024, 1)
            reshaped_pooled_features_np = np.expand_dims(pooled_features_np, axis=-1)

            # Get Label ID
            label_id = label2id[emotion_label]
            label_id_np = np.array(label_id, dtype=np.int32) # Shape ()

            # Save the reshaped pooled feature and label
            feature_filepath = os.path.join(output_dir, f"{unique_id}_features.npy")
            label_filepath = os.path.join(output_dir, f"{unique_id}_label.npy")

            np.save(feature_filepath, reshaped_pooled_features_np) # Save the reshaped features
            np.save(label_filepath, label_id_np)

        except FileNotFoundError:
            print(f"Warning: Audio file not found at {audio_path}. Skipping and not saving.")
            continue # Skip to the next sample
        except Exception as e:
            print(f"Warning: An error occurred processing {audio_path}: {e}. Skipping and not saving.")
            continue # Skip to the next sample

# Extract and save features for training and validation data
extract_and_save_features(train_df, TRAIN_FEATURES_DIR, feature_extractor, wav2vec2_model, label2id, TARGET_SAMPLING_RATE, MAX_LENGTH, device)
extract_and_save_features(val_df, VAL_FEATURES_DIR, feature_extractor, wav2vec2_model, label2id, TARGET_SAMPLING_RATE, MAX_LENGTH, device)

print("\nFeature extraction and saving complete!")

# --- Create tf.data.Dataset from Saved Features ---
print("\nCreating tf.data.Dataset from saved features...")

# --- FIX #2: Corrected numpy loading function ---
# Function to load a single feature and label file using numpy
def _load_feature_label_numpy(feature_path_tensor, label_path_tensor):
    """
    This function is wrapped by tf.py_function.
    It receives EagerTensors, converts them to Python strings,
    and then loads the .npy files.
    """
    # Convert EagerTensor (bytes) to Python string
    feature_path_str = feature_path_tensor.numpy().decode('utf-8')
    label_path_str = label_path_tensor.numpy().decode('utf-8')

    # Load data using the Python string paths
    feature = np.load(feature_path_str)
    label = np.load(label_path_str)
    return feature, label

# Function to wrap the numpy loading function for tf.data.Dataset
def load_feature_label(feature_path, label_path):
    """
    Maps the _load_feature_label_numpy function to the dataset.
    """
    # tf.py_function will execute _load_feature_label_numpy in eager mode
    # The tensor inputs (feature_path, label_path) will be converted to NumPy arrays (strings)
    feature, label = tf.py_function(
        _load_feature_label_numpy,
        [feature_path, label_path],
        [tf.float32, tf.int32] # Specify output types
    )

    # Ensure the shape of the feature is correctly set after loading
    # This uses the 'pooled_feature_dimension' variable defined earlier
    feature.set_shape((pooled_feature_dimension, 1))
    label.set_shape(()) # Label is a scalar

    return feature, label

# Get lists of feature and label file paths
def get_feature_label_paths(directory):
    feature_paths = sorted([os.path.join(directory, f) for f in os.listdir(directory) if f.endswith('_features.npy')])
    label_paths = sorted([os.path.join(directory, f) for f in os.listdir(directory) if f.endswith('_label.npy')])
    # Assumes sorting aligns features and labels correctly (which it should, given the naming convention)
    return feature_paths, label_paths

train_feature_paths, train_label_paths = get_feature_label_paths(TRAIN_FEATURES_DIR)
val_feature_paths, val_label_paths = get_feature_label_paths(VAL_FEATURES_DIR)

# Create datasets from file paths
train_paths_dataset = tf.data.Dataset.from_tensor_slices((train_feature_paths, train_label_paths))
val_paths_dataset = tf.data.Dataset.from_tensor_slices((val_feature_paths, val_label_paths))

# Map the loading function to the datasets
train_dataset = train_paths_dataset.map(load_feature_label, num_parallel_calls=tf.data.AUTOTUNE)
val_dataset = val_paths_dataset.map(load_feature_label, num_parallel_calls=tf.data.AUTOTUNE)

# Apply batching, shuffling, and prefetching
BATCH_SIZE = 32 # You can adjust this
train_dataset = train_dataset.shuffle(buffer_size=len(train_feature_paths)).batch(BATCH_SIZE).repeat().prefetch(buffer_size=tf.data.AUTOTUNE)
val_dataset = val_dataset.batch(BATCH_SIZE).prefetch(buffer_size=tf.data.AUTOTUNE) # No shuffle or repeat for validation

print("✅ tf.data.Dataset pipelines from saved features are ready.")

# Optional: Take a batch to verify the shape and data types
try:
    for batch_features, batch_labels in train_dataset.take(1):
        print("\n--- Verification of a Sample Batch ---")
        print("Sample Batch Features Shape (from new dataset):", batch_features.shape)
        print("Sample Batch Labels Shape (from new dataset):", batch_labels.shape)
        print(f"(Batch size, pooled_feature_dimension, channels) -> ({BATCH_SIZE}, {pooled_feature_dimension}, 1)")
        print("Sample Batch Features Dtype (from new dataset):", batch_features.dtype)
        print("Sample Batch Labels Dtype (from new dataset):", batch_labels.dtype)
        break # Only take one batch for inspection
except Exception as e:
    print(f"\nCould not verify batch shapes. Error: {e}")
    print("This might happen if the train_dataset is empty.")

Starting feature extraction and saving...
Successfully set pooled_feature_dimension from model config: 1024
Extracting features for 750 samples into /content/train_features...


Extracting Features:   0%|          | 0/750 [00:00<?, ?it/s]

Extracting features for 188 samples into /content/val_features...


Extracting Features:   0%|          | 0/188 [00:00<?, ?it/s]


Feature extraction and saving complete!

Creating tf.data.Dataset from saved features...
✅ tf.data.Dataset pipelines from saved features are ready.

--- Verification of a Sample Batch ---
Sample Batch Features Shape (from new dataset): (32, 1024, 1)
Sample Batch Labels Shape (from new dataset): (32,)
(Batch size, pooled_feature_dimension, channels) -> (32, 1024, 1)
Sample Batch Features Dtype (from new dataset): <dtype: 'float32'>
Sample Batch Labels Dtype (from new dataset): <dtype: 'int32'>


In [ ]:
import os
import shutil

def delete_folder(folder_path):
  """Deletes a folder and its contents."""
  try:
    shutil.rmtree(folder_path)
    print(f"Folder '{folder_path}' deleted successfully.")
  except FileNotFoundError:
    print(f"Error: Folder '{folder_path}' not found.")
  except OSError as e:
    print(f"Error deleting folder '{folder_path}': {e}")

folder1 = "/content/train_features"
folder2= "/content/val_features"
delete_folder(folder1)
delete_folder(folder2)

Error: Folder '/content/train_features' not found.
Folder '/content/val_features' deleted successfully.


# **NEW**

In [ ]:
import numpy as np
import librosa
import torch
from tqdm.auto import tqdm
import pandas as pd # Make sure pandas is imported if needed
import tensorflow as tf # Needed for to_tensor later
import random # Import random for augmentation probability


print("Starting feature extraction for all data (without mean pooling)...")

# Ensure necessary variables are defined from previous cells
# TARGET_SAMPLING_RATE, MAX_LENGTH, feature_extractor, wav2vec2_model, label2id, device, pooled_feature_dimension
if 'TARGET_SAMPLING_RATE' not in globals():
    TARGET_SAMPLING_RATE = 16000
    print(f"TARGET_SAMPLING_RATE not found, setting to default: {TARGET_SAMPLING_RATE}")
if 'MAX_LENGTH' not in globals():
    # Assuming feature_extractor is available to get sampling_rate
    if 'feature_extractor' in globals() and feature_extractor is not None:
        MAX_LENGTH = feature_extractor.sampling_rate * 5 # Assuming 5 seconds
    else:
         MAX_LENGTH = 16000 * 5 # Default if feature_extractor is not available
         print("Warning: feature_extractor not found, setting default MAX_LENGTH.")
    print(f"MAX_LENGTH not found, setting to default: {MAX_LENGTH}")

if 'feature_extractor' not in globals() or feature_extractor is None:
     print("Error: feature_extractor is not defined. Please run the cell that defines it.")
     exit()
if 'wav2vec2_model' not in globals() or wav2vec2_model is None:
     print("Error: wav2vec2_model is not defined. Please run the cell that defines it.")
     exit()
if 'label2id' not in globals():
     print("Error: label2id is not defined. Please run the cell that defines it.")
     exit()
if 'device' not in globals():
     device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
     print(f"Device not previously defined, setting to: {device}")
if 'pooled_feature_dimension' not in globals():
     # Assuming the hidden_size of Wav2Vec2 is 1024 based on previous runs
    pooled_feature_dimension = 768
    print(f"pooled_feature_dimension not previously defined, assuming: {pooled_feature_dimension}")

# Ensure dataframes are loaded
if 'train_df' not in globals() or 'val_df' not in globals():
    print("Error: train_df or val_df not found. Please run the cell that loads and splits the data.")
    exit()

#  Data Augmentation Functions

# Noise augmentation (User's Code)
def add_noise(data):
    noise_amp = 0.035 * np.random.uniform() * np.amax(data)
    data = data + noise_amp * np.random.normal(size=data.shape[0])
    return data

# Pitch augmentation (User's Code)
def pitch_shift(data, sampling_rate, pitch_factor=0.7):
    return librosa.effects.pitch_shift(y=data, sr=sampling_rate, n_steps=pitch_factor)

# Time Shifting augmentation (My Code)
def time_shift(data, sampling_rate, shift_limit_ms=100):
    """Shifts audio data in time."""
    shift_limit_samples = int(sampling_rate * shift_limit_ms / 1000)
    shift_amount = random.randint(-shift_limit_samples, shift_limit_samples)
    return np.roll(data, shift_amount)


# Set the Wav2Vec2 model to evaluation mode
wav2vec2_model.eval()

def extract_features_to_arrays(dataframe, feature_extractor, wav2vec2_model, label2id, target_sampling_rate, max_length, device, is_training=False):
    """
    Extracts features using Wav2Vec2 without mean pooling and stores them in NumPy arrays.
    Includes data augmentation for training data.
    """
    features_list = []
    labels_list = []

    print(f"Extracting features for {len(dataframe)} samples...")
    # Use tqdm for a progress bar
    for index, row in tqdm(dataframe.iterrows(), total=len(dataframe), desc="Extracting Features"):
        audio_path = row['filename']
        emotion_label = row['emotion']

        try:
            # Load Audio AND Resample to TARGET_SAMPLING_RATE
            audio_data, sampling_rate = librosa.load(audio_path, sr=target_sampling_rate)

            # List to hold original and augmented data
            augmented_data_list = [audio_data] # Start with the original data

            # Apply all three augmentations for training data and add to the list
            if is_training:
                # Apply noise augmentation
                augmented_data_list.append(add_noise(audio_data.copy())) # Use .copy() to avoid modifying original

                # Apply pitch shift augmentation
                # Randomly choose a pitch shift factor for pitch shifting
                pitch_factor_rand = random.uniform(-4, 4)
                augmented_data_list.append(pitch_shift(audio_data.copy(), sampling_rate, pitch_factor_rand))

                # Apply time shift augmentation

                time_shift_limit_ms = random.uniform(50, 200)
                augmented_data_list.append(time_shift(audio_data.copy(), sampling_rate, time_shift_limit_ms))



            for current_audio_data in augmented_data_list:
                # Apply Feature Extractor
                features = feature_extractor(
                    current_audio_data,
                    sampling_rate=sampling_rate,
                    padding='max_length',
                    max_length=max_length,
                    truncation=True,
                    return_tensors="pt"
                )
                input_values = features['input_values'].to(device) # Move input to device

                # Pass input through the full Wav2Vec2 model
                with torch.no_grad():
                    model_outputs = wav2vec2_model(input_values)

                # Extract the last hidden state (rich features)
                raw_features = model_outputs.last_hidden_state # Shape (batch_size=1, sequence_length', hidden_size)

                # Convert PyTorch tensor to NumPy and squeeze the batch dimension (1,)

                processed_features_np = raw_features.squeeze(0).cpu().numpy().astype(np.float32)

                # --- No Mean Pooling ---


                # Get Label ID
                label_id = label2id[emotion_label]
                label_id_np = np.array(label_id, dtype=np.int32) # Shape ()

                # Append the full sequence of features and label
                features_list.append(processed_features_np)
                labels_list.append(label_id_np)

        except FileNotFoundError:
            print(f"Warning: Audio file not found at {audio_path}. Skipping.")
            continue # Skip to the next sample
        except Exception as e:
            print(f"Warning: An error occurred processing {audio_path}: {e}. Skipping.")
            continue # Skip to the next sample

    # Convert lists to NumPy arrays

    features_array = np.array(features_list)
    labels_array = np.array(labels_list)

    return features_array, labels_array

try:

    dummy_audio = np.zeros(MAX_LENGTH, dtype=np.float32)
    dummy_features = feature_extractor(
        dummy_audio,
        sampling_rate=TARGET_SAMPLING_RATE,
        padding='max_length',
        max_length=MAX_LENGTH,
        truncation=True,
        return_tensors="pt"
    )
    dummy_input_values = dummy_features['input_values'].to(device)
    with torch.no_grad():
        dummy_model_outputs = wav2vec2_model(dummy_input_values)

    expected_sequence_length = dummy_model_outputs.last_hidden_state.shape[1]
    print(f"Expected sequence length after feature extraction: {expected_sequence_length}")

    extracted_feature_shape = (expected_sequence_length, pooled_feature_dimension) # Shape is (sequence_length, hidden_size)
    print(f"Expected feature shape per sample (without pooling): {extracted_feature_shape}")

except Exception as e:
    print(f"Error determining expected sequence length: {e}")
    print("Assuming a default sequence length for now. This may cause shape mismatch errors.")
    expected_sequence_length = 250
    extracted_feature_shape = (expected_sequence_length, pooled_feature_dimension)
    print(f"Assuming feature shape per sample (without pooling): {extracted_feature_shape}")


# Extract features for training and validation data

train_features_np, train_labels_np = extract_features_to_arrays(
    train_df, feature_extractor, wav2vec2_model, label2id,
    TARGET_SAMPLING_RATE, MAX_LENGTH, device, is_training=True
)

val_features_np, val_labels_np = extract_features_to_arrays(
    val_df, feature_extractor, wav2vec2_model, label2id,
    TARGET_SAMPLING_RATE, MAX_LENGTH, device, is_training=False
)


print("\nFeature extraction complete!")
print(f"Training features shape: {train_features_np.shape}")
print(f"Training labels shape: {train_labels_np.shape}")
print(f"Validation features shape: {val_features_np.shape}")
print(f"Validation labels shape: {val_labels_np.shape}")



Starting feature extraction for all data (without mean pooling)...
TARGET_SAMPLING_RATE not found, setting to default: 16000
MAX_LENGTH not found, setting to default: 80000
pooled_feature_dimension not previously defined, assuming: 768
Expected sequence length after feature extraction: 249
Expected feature shape per sample (without pooling): (249, 768)
Extracting features for 750 samples...


Extracting Features:   0%|          | 0/750 [00:00<?, ?it/s]

Extracting features for 188 samples...


Extracting Features:   0%|          | 0/188 [00:00<?, ?it/s]


Feature extraction complete!
Training features shape: (3000, 249, 768)
Training labels shape: (3000,)
Validation features shape: (188, 249, 768)
Validation labels shape: (188,)


In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping,ReduceLROnPlateau
model_checkpoint = ModelCheckpoint('0603)best_model1_weights.keras', monitor='val_accuracy', save_best_only=True)
early_stop=EarlyStopping(monitor='val_accuracy',mode='auto',patience=5,restore_best_weights=True)
lr_reduction=ReduceLROnPlateau(monitor='val_accuracy',patience=3,verbose=1,factor=0.5,min_lr=0.00001)

# **1D CNN WITH ATTENTION**

In [ ]:
import keras
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM,BatchNormalization , GRU
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from keras.layers import Input, Flatten, Dropout, Activation
from keras.layers import Conv1D, MaxPooling1D, AveragePooling1D
from keras.models import Model
from keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import SGD

In [ ]:
import tensorflow as tf
import tensorflow.keras.layers as L
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l2

# Channel Attention Layer for 1D (Corrected Implementation)
class ChannelAttention(L.Layer):
    def __init__(self, reduction_ratio=16, **kwargs):
        super(ChannelAttention, self).__init__(**kwargs)
        self.reduction_ratio = reduction_ratio

    def build(self, input_shape):
        self.channels = input_shape[-1]
        if self.channels is None:
             raise ValueError("Channel dimension must be known to build ChannelAttention layer.")

        # Define Dense layers here, they will be built automatically on first call
        self.avg_dense1 = L.Dense(self.channels // self.reduction_ratio, activation='relu', kernel_initializer='he_normal', use_bias=True, bias_initializer='zeros')
        self.avg_dense2 = L.Dense(self.channels, kernel_initializer='he_normal', use_bias=True, bias_initializer='zeros')
        self.max_dense1 = L.Dense(self.channels // self.reduction_ratio, activation='relu', kernel_initializer='he_normal', use_bias=True, bias_initializer='zeros')
        self.max_dense2 = L.Dense(self.channels, kernel_initializer='he_normal', use_bias=True, bias_initializer='zeros')

        super(ChannelAttention, self).build(input_shape)


    def call(self, inputs):
        avg_pool = tf.reduce_mean(inputs, axis=1, keepdims=True)
        max_pool = tf.reduce_max(inputs, axis=1, keepdims=True)

        # The Dense layers will be built here on the first call of the ChannelAttention layer
        avg_out = self.avg_dense1(avg_pool)
        avg_out = self.avg_dense2(avg_out)

        max_out = self.max_dense1(max_pool)
        max_out = self.max_dense2(max_out)

        attention = tf.nn.sigmoid(avg_out + max_out)

        return inputs * attention

# Spatial Attention Layer for 1D
class SpatialAttention(L.Layer):
    def __init__(self, **kwargs):
        super(SpatialAttention, self).__init__(**kwargs)

    def build(self, input_shape):
        # Keeping the simpler build as Conv1D can handle symbolic shapes.
        self.conv = L.Conv1D(1, kernel_size=7, padding='same', activation='sigmoid', kernel_initializer='he_normal', use_bias=False)
        super(SpatialAttention, self).build(input_shape)

    def call(self, inputs):
        avg_pool = tf.reduce_mean(inputs, axis=-1, keepdims=True)
        max_pool = tf.reduce_max(inputs, axis=-1, keepdims=True)
        concat = tf.concat([avg_pool, max_pool], axis=-1)
        attention = self.conv(concat)
        return inputs * attention

# Define the model

# The input shape should now match the pre-extracted features (sequence_length, hidden_size)
# as modified in the feature extraction cell (10a8af35)
input_shape = (expected_sequence_length, pooled_feature_dimension) # Use the variables defined earlier
inputs = tf.keras.Input(shape=input_shape) # Input layer now directly accepts the sequence

# Convolutional layers
# Reduced filters and added more regularization
x = L.Conv1D(128, kernel_size=7, strides=1, activation='relu', kernel_regularizer=l2(0.005))(inputs) # Increased L2
x = L.BatchNormalization(axis=-1)(x)
x = L.MaxPool1D(pool_size=4, strides=4)(x)

x = L.Conv1D(128, kernel_size=7, strides=1, activation='relu', kernel_regularizer=l2(0.005))(x) # Increased L2
x = L.BatchNormalization(axis=-1)(x)
x = L.MaxPool1D(pool_size=4, strides=4)(x)
x = L.Dropout(0.4)(x) # Increased Dropout

# Channel Attention
channel_attention = ChannelAttention()
x = channel_attention(x)

# Spatial Attention
spatial_attention = SpatialAttention()
x = spatial_attention(x)

# Fully connected layers
x = L.Flatten()(x)
x = L.Dense(64, activation='relu', kernel_regularizer=l2(0.005))(x) # Increased L2
x = L.BatchNormalization(axis=-1)(x)
x = L.Dropout(0.4)(x) # Added Dropout

# Reduced complexity by potentially having fewer units or layers if needed, but primarily increased regularization here.
# If further complexity reduction is needed, we can remove a Conv1D block or reduce Dense units.
outputs = L.Dense(num_labels, activation='softmax')(x)

# Create model
model = Model(inputs, outputs)

# Compile model
optimizer = tf.keras.optimizers.Adam(learning_rate=0.00001)
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "functional_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_8 (InputLayer)      │ (None, 249, 1024)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_10 (Conv1D)              │ (None, 243, 128)       │       917,632 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_10          │ (None, 243, 128)       │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_8 (MaxPooling1D)  │ (None, 60, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_11 (Conv1D)              │ (None, 54, 128)        │       114,816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_11          │ (None, 54, 128)        │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_9 (MaxPooling1D)  │ (None, 13, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ (None, 13, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ channel_attention_4             │ (None, 13, 128)        │         4,368 │
│ (ChannelAttention)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ spatial_attention_4             │ (None, 13, 128)        │            14 │
│ (SpatialAttention)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 1664)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ (None, 64)             │       106,560 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_12          │ (None, 64)             │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_9 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_17 (Dense)                │ (None, 4)              │           260 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,144,930 (4.37 MB)

 Trainable params: 1,144,290 (4.37 MB)

 Non-trainable params: 640 (2.50 KB)

In [ ]:
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau


history = model.fit(
    train_features_np,
    train_labels_np,
    validation_data=(val_features_np, val_labels_np),
    epochs=200,
    batch_size=16, # Reduced batch_size further
)

Epoch 1/200
188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.2832 - loss: 4.5281 - val_accuracy: 0.2926 - val_loss: 3.9553
Epoch 2/200
188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.2659 - loss: 4.5920 - val_accuracy: 0.2606 - val_loss: 4.0236
Epoch 3/200
188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.2615 - loss: 4.5743 - val_accuracy: 0.2606 - val_loss: 4.0654
Epoch 4/200
188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.2756 - loss: 4.5509 - val_accuracy: 0.2447 - val_loss: 4.0787
Epoch 5/200
188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.2607 - loss: 4.6194 - val_accuracy: 0.2553 - val_loss: 4.0743
Epoch 6/200
188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.2596 - loss: 4.5945 - val_accuracy: 0.2394 - val_loss: 4.0706
Epoch 7/200
188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.2745 - loss: 4.5892 - val_accuracy: 0.2394 - val_loss: 4.0740
Epoch 8/200
188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.2899 - loss: 4.5132 - val_accu

KeyboardInterrupt: 

# **FEED FORWARD NEURAL NETWORK**

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout, Input, Flatten, Conv1D, MaxPooling1D
from tensorflow.keras.regularizers import l2

print("Defining a dense and complex 1D CNN model...")

input_shape_cnn = (expected_sequence_length, pooled_feature_dimension)

cnn_model_ffn_slot = Sequential([
    Input(shape=input_shape_cnn),

    # First Convolutional Block (Dense)
    Conv1D(filters=256, kernel_size=7, strides=1, activation='relu', kernel_regularizer=l2(0.005), padding='same'),
    BatchNormalization(),
    MaxPooling1D(pool_size=4, strides=4, padding='same'),


    # Second Convolutional Block (Complex)
    Conv1D(filters=256, kernel_size=5, strides=1, activation='relu', kernel_regularizer=l2(0.005), padding='same'),
    BatchNormalization(),
    MaxPooling1D(pool_size=4, strides=4, padding='same'),


    # Third Convolutional Block (More Complex)
    Conv1D(filters=128, kernel_size=3, strides=1, activation='relu', kernel_regularizer=l2(0.005), padding='same'),
    BatchNormalization(),
    MaxPooling1D(pool_size=2, strides=2, padding='same'),


    Flatten(),

    # Dense Layers (Dense and Complex)
    Dense(512, activation='relu', kernel_regularizer=l2(0.005)),
    BatchNormalization(),
    Dropout(0.1),

    Dense(256, activation='relu', kernel_regularizer=l2(0.005)),
    BatchNormalization(),
    Dropout(0.1),

    Dense(128, activation='relu', kernel_regularizer=l2(0.005)),
    BatchNormalization(),
    Dropout(0.1),

    Dense(num_labels, activation='softmax')
])

optimizer = tf.keras.optimizers.Adam(learning_rate=0.00001)
cnn_model_ffn_slot.compile(optimizer=optimizer,
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

cnn_model_ffn_slot.summary()

Defining a dense and complex 1D CNN model...


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_3 (Conv1D)               │ (None, 249, 256)       │     1,376,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_6           │ (None, 249, 256)       │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_3 (MaxPooling1D)  │ (None, 63, 256)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_4 (Conv1D)               │ (None, 63, 256)        │       327,936 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_7           │ (None, 63, 256)        │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_4 (MaxPooling1D)  │ (None, 16, 256)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_5 (Conv1D)               │ (None, 16, 128)        │        98,432 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_8           │ (None, 16, 128)        │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_5 (MaxPooling1D)  │ (None, 8, 128)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 512)            │       524,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_9           │ (None, 512)            │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_10          │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_11          │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 4)              │           516 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,498,564 (9.53 MB)

 Trainable params: 2,495,492 (9.52 MB)

 Non-trainable params: 3,072 (12.00 KB)

In [ ]:
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

print("Starting training of the Feed-Forward Network with pre-extracted features (NumPy arrays)...")


ffn_early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
ffn_lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5)

ffn_callbacks = [ffn_early_stopping, ffn_lr_scheduler]

ffn_history = cnn_model_ffn_slot.fit(
    train_features_np,
    train_labels_np,
    validation_data=(val_features_np, val_labels_np),
    epochs=80,

)



Starting training of the Feed-Forward Network with pre-extracted features (NumPy arrays)...


# **1D CNN**

# **TESTING**

In [ ]:
import pandas as pd
import os
import tensorflow as tf
import numpy as np
import librosa
import torch
from tqdm.auto import tqdm

# **GOOGLE DRIVE**

In [ ]:
TEST_CSV_PATH = "/content/drive/MyDrive/DeepLearning/SER/Twi_Sampled.csv" # <-- Replace with the path to your test CSV file
TEST_AUDIO_FOLDER = "/content/drive/MyDrive/SER/DeepLearning/Twi/" # <-- Replace with the path to the folder containing your test audio files

# **ARC LAB**

In [ ]:
import pandas as pd
import os

# --- 1. Define Paths ---
TEST_CSV_RUNTIME_PATH = "/content/Twi_Sampled.csv" # Updated CSV name
TEST_AUDIO_FOLDER = "/content/test/Twi/" # Folder where audio was unzipped
NEW_TEST_CSV_RUNTIME_PATH = "/content/updated_sampled_data_runtime.csv"


print(f"Loading test CSV from: {TEST_CSV_RUNTIME_PATH}")
print(f"Test audio folder is: {TEST_AUDIO_FOLDER}")
print(f"New test CSV will be saved to: {NEW_TEST_CSV_RUNTIME_PATH}")

# --- 2. Load the Test CSV ---
try:
    test_df = pd.read_csv(TEST_CSV_RUNTIME_PATH)
    print("✅ Successfully loaded Sampled_Data.csv.")
    print("Original Test DataFrame head:")
    display(test_df.head())
except FileNotFoundError:
    print(f"Error: Test CSV file not found at {TEST_CSV_RUNTIME_PATH}. Please ensure you have uploaded 'Sampled_Data.csv' to your runtime.")
    exit()
except Exception as e:
    print(f"An error occurred while loading the test CSV: {e}")
    exit()

# --- 3. Update File Paths in Test DataFrame ---
print("\nUpdating file paths in the test DataFrame...")
def update_test_runtime_filepath(filename):
    """
    Updates the file path to point to the specified runtime test audio folder.
    Assumes the original filename is just the base filename.
    We extract the base filename and prepend the runtime test audio folder path.
    """
    base_filename = os.path.basename(filename)
    new_filepath = os.path.join(TEST_AUDIO_FOLDER, base_filename)
    return new_filepath

try:
    # Ensure the column name is correct based on Sampled_Data.csv
    # If the column with filenames is not 'filename', you'll need to change it here.
    if 'filename' in test_df.columns:
        test_df['filename'] = test_df['filename'].apply(update_test_runtime_filepath)
        print("✅ File paths in the test DataFrame updated.")
        print("Test DataFrame head with updated paths:")
        display(test_df.head())
    else:
        print("Error: 'filename' column not found in the test DataFrame. Please check your CSV file.")
        exit()
except KeyError:
    print("Error: 'filename' column not found in the test DataFrame after checking. This should not happen.")
    exit()
except Exception as e:
    print(f"An unexpected error occurred while updating test DataFrame paths: {e}")
    exit()

# --- 4. Save New Test CSV ---
print(f"\nSaving updated Test DataFrame to {NEW_TEST_CSV_RUNTIME_PATH}...")
try:
    test_df.to_csv(NEW_TEST_CSV_RUNTIME_PATH, index=False)
    print(f"✅ Successfully saved updated test CSV to {NEW_TEST_CSV_RUNTIME_PATH}")
except Exception as e:
    print(f"An error occurred while saving the new test CSV: {e}")
    exit()

# --- 5. Verify New Test CSV (Optional) ---
print("\nVerifying the newly saved test CSV...")
try:
    verify_test_df = pd.read_csv(NEW_TEST_CSV_RUNTIME_PATH)
    print("✅ Successfully loaded the new test CSV for verification.")
    print("New Test CSV head:")
    display(verify_test_df.head())
except FileNotFoundError:
    print(f"Error: Newly saved test CSV file not found at {NEW_TEST_CSV_RUNTIME_PATH}. Saving may have failed.")
except Exception as e:
    print(f"An error occurred while verifying the new test CSV: {e}")

print("\nTest data preparation complete.")

Loading test CSV from: /content/Twi_Sampled.csv
Test audio folder is: /content/test/Twi/
New test CSV will be saved to: /content/updated_sampled_data_runtime.csv
✅ Successfully loaded Sampled_Data.csv.
Original Test DataFrame head:


,Name,Emotion
0,Happy-Twi-Prince.m4a,Happy
1,Sadness-Twi-Prince.m4a,Sad
2,Neutral-Twi-Prince.m4a,Neutral
3,Happy-Twi-Prince 2.m4a,Happy
4,Angry-Twi-Shifa-4.m4a,Anger



Updating file paths in the test DataFrame...
Error: 'filename' column not found in the test DataFrame. Please check your CSV file.

Saving updated Test DataFrame to /content/updated_sampled_data_runtime.csv...
✅ Successfully saved updated test CSV to /content/updated_sampled_data_runtime.csv

Verifying the newly saved test CSV...
✅ Successfully loaded the new test CSV for verification.
New Test CSV head:


,Name,Emotion
0,Happy-Twi-Prince.m4a,Happy
1,Sadness-Twi-Prince.m4a,Sad
2,Neutral-Twi-Prince.m4a,Neutral
3,Happy-Twi-Prince 2.m4a,Happy
4,Angry-Twi-Shifa-4.m4a,Anger



Test data preparation complete.


In [ ]:
TEST_AUDIO_FOLDER = "/content/test/"
TEST_CSV_PATH = "/content/updated_sampled_data_runtime.csv"

# **CONTINUATION**

In [ ]:
# --- User Input: Define Paths ---
# Please replace these with the actual paths to your test data




print(f"Using test CSV path: {TEST_CSV_PATH}")
print(f"Using test audio folder path: {TEST_AUDIO_FOLDER}")

# --- 1. Load New Test Data ---
print("\nLoading new test dataset...")
try:
    test_df = pd.read_csv(TEST_CSV_PATH)
    print(f"✅ Successfully loaded test data from {TEST_CSV_PATH}")
    print(f"Number of test samples: {len(test_df)}")
    display(test_df.head())
except FileNotFoundError:
    print(f"Error: Test CSV file not found at {TEST_CSV_PATH}. Please check the path.")
    # Exit the cell if the CSV is not found
    exit()
except Exception as e:
    print(f"An error occurred while loading the test CSV: {e}")
    exit()


# --- 2. Update File Paths in Test DataFrame ---
print("\nUpdating file paths in the test DataFrame...")
def update_test_filepath(filename):
    """
    Updates the file path to point to the specified test audio folder.
    Assumes the original filename is just the base filename.
    """
    base_filename = os.path.basename(filename)
    new_filepath = os.path.join(TEST_AUDIO_FOLDER, base_filename)
    return new_filepath

try:
    test_df['filename'] = test_df['filename'].apply(update_test_filepath)
    print("✅ File paths in the test DataFrame updated.")
    display(test_df.head()) # Display the first few rows to verify the updated paths
except KeyError:
    print("Error: 'filename' column not found in the test DataFrame. Please check your test CSV file.")
    exit()
except Exception as e:
    print(f"An unexpected error occurred while updating test DataFrame paths: {e}")
    exit()


Using test CSV path: /content/updated_sampled_data_runtime.csv
Using test audio folder path: /content/test/

Loading new test dataset...
✅ Successfully loaded test data from /content/updated_sampled_data_runtime.csv
Number of test samples: 19


,Name,Emotion
0,Happy-Twi-Prince.m4a,Happy
1,Sadness-Twi-Prince.m4a,Sad
2,Neutral-Twi-Prince.m4a,Neutral
3,Happy-Twi-Prince 2.m4a,Happy
4,Angry-Twi-Shifa-4.m4a,Anger



Updating file paths in the test DataFrame...
Error: 'filename' column not found in the test DataFrame. Please check your test CSV file.


In [ ]:
# --- 3. Process Test Data (using the same generator logic as before) ---
print("\nProcessing test data for evaluation...")

# Ensure necessary variables are defined from previous cells
# These should be available if the preceding cells were run successfully
# TARGET_SAMPLING_RATE, MAX_LENGTH, feature_extractor, wav2vec2_model, label2id, device, pooled_feature_dimension
if 'TARGET_SAMPLING_RATE' not in globals():
    print("Error: TARGET_SAMPLING_RATE is not defined. Please run the cell that defines it.")
    exit()
if 'MAX_LENGTH' not in globals():
    print("Error: MAX_LENGTH is not defined. Please run the cell that defines it.")
    exit()
if 'feature_extractor' not in globals():
    print("Error: feature_extractor is not defined. Please run the cell that defines it.")
    exit()
if 'wav2vec2_model' not in globals():
    print("Error: wav2vec2_model is not defined. Please run the cell that defines it.")
    exit()
if 'label2id' not in globals():
    print("Error: label2id is not defined. Please run the cell that defines it.")
    exit()
if 'device' not in globals():
     # Attempt to define device if not already defined
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Device not previously defined, setting to: {device}")
if 'pooled_feature_dimension' not in globals():
     # Attempt to define pooled_feature_dimension if not already defined
     # Assuming the hidden_size of Wav2Vec2 is 1024 based on previous runs
    pooled_feature_dimension = 1024
    print(f"pooled_feature_dimension not previously defined, assuming: {pooled_feature_dimension}")


# Reuse the data_generator function defined in cell 0dbf9d5d
# Assuming the data_generator function is available in the current kernel state.
# If not, you would need to redefine it here or ensure the cell defining it is run first.
# For this response, we assume it's available.

# Define the feature shape after pooling and reshaping for the tf.data.Dataset
feature_shape_after_pooling = (pooled_feature_dimension, 1) # Shape is (hidden_size, 1)

# Create the test tf.data.Dataset object
# Use a batch size appropriate for evaluation (can be larger than training batch size)
TEST_BATCH_SIZE = 64 # Adjust as needed
test_dataset = create_tf_dataset(
    test_df,
    feature_extractor,
    wav2vec2_model,
    label2id,
    TARGET_SAMPLING_RATE,
    MAX_LENGTH,
    TEST_BATCH_SIZE,
    device,
    feature_shape_after_pooling,
    is_training=False # Important: Do not shuffle or repeat for evaluation
)

print("✅ Test tf.data.Dataset pipeline is ready.")

In [ ]:
# --- 4. Evaluate CNN Model ---
print("\nEvaluating CNN model on test data...")
if 'model' in globals() and model is not None:
    try:
        cnn_evaluation_results = model.evaluate(test_dataset)
        print("✅ CNN Model Evaluation Results:")
        print(f"  Test Loss: {cnn_evaluation_results[0]:.4f}")
        print(f"  Test Accuracy: {cnn_evaluation_results[1]:.4f}")
    except Exception as e:
        print(f"Error during CNN model evaluation: {e}")
else:
    print("Warning: CNN model ('model') not found. Skipping CNN evaluation.")


# --- 5. Evaluate FFN Model ---
print("\nEvaluating Feed-Forward Network model on test data...")
if 'ffn_model' in globals() and ffn_model is not None:
    try:
        # Ensure the FFN model expects the correct input shape (batch_size, hidden_size)
        # The test_dataset yields batches of shape (batch_size, hidden_size, 1)
        # We need to remove the last dimension for the FFN input
        test_dataset_ffn_input = test_dataset.map(lambda x, y: (tf.squeeze(x, axis=-1), y))

        ffn_evaluation_results = ffn_model.evaluate(test_dataset_ffn_input)
        print("✅ Feed-Forward Network Model Evaluation Results:")
        print(f"  Test Loss: {ffn_evaluation_results[0]:.4f}")
        print(f"  Test Accuracy: {ffn_evaluation_results[1]:.4f}")
    except Exception as e:
        print(f"Error during FFN model evaluation: {e}")
else:
    print("Warning: Feed-Forward Network model ('ffn_model') not found. Skipping FFN evaluation.")

print("\n🎉 Evaluation on test data complete.")

In [ ]:
import os

# Define paths to save the models in the runtime /content/ directory
CNN_MODEL_SAVE_PATH = "/content/cnn_model.keras"
FFN_MODEL_SAVE_PATH = "/content/ffn_model.keras"

print(f"Saving CNN model to: {CNN_MODEL_SAVE_PATH}")
try:
    # Ensure the CNN model exists before saving
    if 'model' in globals() and model is not None:
        model.save(CNN_MODEL_SAVE_PATH)
        print("✅ CNN model saved successfully.")
    else:
        print("Warning: CNN model ('model') not found. Skipping CNN model saving.")
except Exception as e:
    print(f"Error saving CNN model: {e}")

print(f"\nSaving Feed-Forward Network model to: {FFN_MODEL_SAVE_PATH}")
try:
    # Ensure the FFN model exists before saving
    if 'ffn_model' in globals() and ffn_model is not None:
        ffn_model.save(FFN_MODEL_SAVE_PATH)
        print("✅ Feed-Forward Network model saved successfully.")
    else:
        print("Warning: Feed-Forward Network model ('ffn_model') not found. Skipping FFN model saving.")
except Exception as e:
    print(f"Error saving Feed-Forward Network model: {e}")

print("\nModel saving process finished.")